In [493]:
import scipy.stats
import numpy as np
import plotly.express as px
import chart_studio.plotly as py
from plotly.offline import iplot

confidence=0.95
n_mca_samples=6
n_voxels=5000

In [494]:
def mean(x):
    return np.mean(x)

def var_biased(x):
    '''
    S^2_{n} = sum((X-mu)^2)/N
    '''
    _mean = mean(x)
    _n = x.size
    _diff = np.sum(np.square(x - _mean))
    return _diff/_n

def var_unbiased(x):
    '''
    S^2_{n-1} = sum((X-mu)^2)/(N-1)
    '''
    _mean = mean(x)
    _n = x.size - 1
    _diff = np.sum(np.square(x - _mean))
    return _diff/_n

def std_biased_1(x):
    '''
    std = sqrt( S^2_n )
    '''
    return np.sqrt(var_biased(x))

def std_biased_2(x):
    '''
    std = sqrt( S^2_{n-1} )
    '''
    return np.sqrt(var_unbiased(x))

def c4(n):
    '''
    c4(n) = sqrt(2/n-1) (gamma(n/2)/gamma(n-1/2))
    '''
    gamma = scipy.special.gamma
    return np.sqrt(2/(n-1)) * (gamma(n/2)/gamma((n-1)/2))

def std_unbiased(x):
    '''
    std = sqrt( S^2_{n-1} ) / c4(n)
    '''
    n = x.size
    _std = std_biased_2(x)/c4(n)
    return _std

def is_interval(x, mean, std, confidence):
    '''
    Check x values are in interval
    alpha = confidence
    (mean - t_{alpha} * (std/sqrt(n)), mean - t_{alpha} * (std/sqrt(n) )
    '''
    n = x.size
    coef_t = scipy.stats.t.interval(alpha=confidence, loc=mean, scale=std, df=n-1)
    success = np.logical_and(coef_t[0] <= x, x <= coef_t[1])
    return success

def get_success(x, mean, std, confidence):
    trials = x.size
    success = is_interval(x, mean, std, confidence)
    nb_success = np.count_nonzero(success)
    nb_fail = trials - nb_success
    return trials, nb_success, nb_fail
    
def draw_samples(n):
    x = np.random.normal(size=n)
    return x

def test(n, std, confidence):    
    x = draw_samples(n)
    _mean = mean(x)
    _std = std(x)
    nb_trials, nb_success, nb_fail = get_success(x, _mean, _std, confidence)
    return nb_success
    
def run(std, n_mca_samples, n_voxels, confidene):
    np.random.seed(0)
    nb_successes = [test(n_mca_samples, std, confidence) for i in range(n_voxels)]
    per_succ = np.sum(nb_successes)/(n_voxels*n_mca_samples) * 100
    return per_succ

def plot(ratio, title):
    histo = px.histogram(ratio, nbins=100, histnorm='percent', range_x=(0,100))
    histo.update_layout(title=title)
    histo.update_xaxes(title='% Success')
    return histo


In [495]:
print(f'Test std_biased_1 for confidence ({confidence})')
for n_mca_samples in [6, 10, 30, 100]:
    ratio = run(std_biased_1, n_mca_samples, n_voxels, confidence)
    print(f'n_voxels: {n_voxels}, n_mca_samples: {n_mca_samples:3}, success ratio: {ratio:.2f}%')

Test std_biased_1 for confidence (0.95)
n_voxels: 5000, n_mca_samples:   6, success ratio: 100.00%
n_voxels: 5000, n_mca_samples:  10, success ratio: 98.76%
n_voxels: 5000, n_mca_samples:  30, success ratio: 96.11%
n_voxels: 5000, n_mca_samples: 100, success ratio: 95.29%


In [496]:
print(f'Test std_biased_2 for confidence ({confidence})')
for n_mca_samples in [6, 10, 30, 100]:
    ratio = run(std_biased_2, n_mca_samples, n_voxels, confidence)
    print(f'n_voxels: {n_voxels}, n_mca_samples: {n_mca_samples:3}, success ratio: {ratio:.2f}%')

Test std_biased_2 for confidence (0.95)
n_voxels: 5000, n_mca_samples:   6, success ratio: 100.00%
n_voxels: 5000, n_mca_samples:  10, success ratio: 99.38%
n_voxels: 5000, n_mca_samples:  30, success ratio: 96.48%
n_voxels: 5000, n_mca_samples: 100, success ratio: 95.40%


In [497]:
print(f'Test std_unbiased for confidence ({confidence})')
for n_mca_samples in [6, 10, 30, 100]:
    ratio = run(std_unbiased, n_mca_samples, n_voxels, confidence)
    print(f'n_voxels: {n_voxels}, n_mca_samples: {n_mca_samples:3}, success ratio: {ratio:.2f}%')
    

Test std_unbiased for confidence (0.95)
n_voxels: 5000, n_mca_samples:   6, success ratio: 100.00%
n_voxels: 5000, n_mca_samples:  10, success ratio: 99.61%
n_voxels: 5000, n_mca_samples:  30, success ratio: 96.67%
n_voxels: 5000, n_mca_samples: 100, success ratio: 95.46%
